## Data Distribution

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from CNN.loader import get_split
from CNN.model import train
    
DATA_DIR = os.path.join('data', 'mammals_calls')
h = 164
w = 397

np.random.seed(2025)

In [ ]:
subfolders = [f.path for f in os.scandir(DATA_DIR) if f.is_dir()]
data_info = {}
for subfolder in subfolders:
    species_name = os.path.basename(subfolder)
    file_count = len([f for f in os.listdir(subfolder) if os.path.isfile(os.path.join(subfolder, f))])
    data_info[species_name] = file_count
count_df = pd.DataFrame(list(data_info.items()), columns=['species', 'file_count'])
count_df = count_df.sort_values(by='file_count', ascending=False)


In [ ]:
count_thousands = count_df[count_df['file_count'] > 1000]
count_hundreds = count_df[(count_df['file_count'] > 100) & (count_df['file_count'] < 1000)]
count_tens = count_df[count_df['file_count'] < 100]

plt.figure(figsize=(15, 6))
plt.bar(count_thousands['species'], count_thousands['file_count'], color="#87CEEB")
plt.bar(count_hundreds['species'], count_hundreds['file_count'], color='#00688B')
plt.bar(count_tens['species'], count_tens['file_count'], color="#191970")
plt.xlabel('Species')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Log Scaled Number of Spectrograms')
plt.yscale("log")
plt.title('Number of Spectrograms per Species')

plt.tight_layout()
plt.show()

print(f'Total number of species: {count_df.shape[0]}')
print(f'Number of species with more than 1000 spectrograms: {count_thousands.shape[0]}')
print(f'Number of species with more than 100 but less than 1000 spectrograms: {count_hundreds.shape[0]}')
print(f'Number of species with less than 100 spectrograms: {count_tens.shape[0]}')


## Training CNN

In [ ]:
dataset = os.listdir(DATA_DIR)
class_list = [f for f in dataset if len(os.listdir(os.path.join(DATA_DIR, f))) > 999 and
              os.path.isdir(os.path.join(DATA_DIR, f))]
n_classes = len(class_list)
print(f'Total classes found: {n_classes}')

split_perc = {'train': 0.6, 'val': 0.4}
split_ds = get_split(DATA_DIR, class_list, split_perc, h, w)

In [ ]:
train(split_ds['train'], split_ds['val'], patience=5,cp_path='checkpoints', checkpoint_freq=5, w_h = (w, h), n_classes=n_classes)